In [ ]:
from datasets import load_dataset

dataset = load_dataset("tner/btc")

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 6338
    })
    validation: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 2000
    })
})


In [31]:
# Get tokens and ner_tags from the training split
train_tokens = dataset["train"]["tokens"]
train_tags = dataset["train"]["tags"]

# Optional: for validation or test
val_tokens = dataset["validation"]["tokens"]
val_tags = dataset["validation"]["tags"]

train_tags


[[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
 [6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6],
 [6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6],
 [6, 6, 6, 6, 6, 6],
 [6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  0,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  2,
  6,
  6,
  6,
  6,
  6,
  6,
  6],
 [6, 6, 6, 6, 6],
 [6, 6, 6, 6, 6, 1, 4, 4, 6, 6, 6],
 [6, 6, 6, 6, 6, 6, 6],
 [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
 [6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
 [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
 [6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6],
 [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
 [6, 6, 2, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
 [6, 6, 6,

In [14]:
!pip install datasets transformers torch seqeval torchcrf

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ------ --------------------------------- 1.0/6.2 MB 6.3 MB/s eta 0:00:01
   --------------- ------------------------ 2.4/6.2 MB 5.8 MB/s eta 0:00:01
   ------------------------- -------------- 3.9/6.2 MB 6.2 MB/s eta 0:00:01
   --------------------------------- ------ 5.2/6.2 MB 6.2 MB/s eta 0:00:01
   ---------------------------------------- 6.2/6.2 MB 5.9 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling 

DEPRECATION: Loading egg at c:\users\arpan\anaconda3\envs\text_analytics\lib\site-packages\huggingface_hub-0.29.1-py3.8.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word_id of None. We set their label to -100 so they're ignored in the loss.
            if word_idx is None:
                label_ids.append(-100)
            # Only label the first token of a given word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                # For subword tokens, you can either label as continuation (I-tag) or ignore
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply the tokenization (example on train split)
tokenized_datasets = dataset["train"].map(tokenize_and_align_labels, batched=True)



Map:   0%|          | 0/6338 [00:00<?, ? examples/s]

In [18]:
import torch
import torch.nn as nn
from torchcrf import CRF

class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim=100, hidden_dim=128):
        super(BiLSTM_CRF, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=1,
                            bidirectional=True, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.crf = CRF(tagset_size, batch_first=True)
    
    def forward(self, x, tags=None, mask=None):
        # x: (batch_size, seq_length)
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        emissions = self.hidden2tag(lstm_out)
        
        if tags is not None:
            # Compute negative log-likelihood
            loss = -self.crf(emissions, tags, mask=mask, reduction='mean')
            return loss
        else:
            # Decode the best sequence
            return self.crf.decode(emissions, mask=mask)


In [ ]:
from torch.utils.data import DataLoader, Dataset
import numpy as np

class NERDataset(Dataset):
    def __init__(self, tokens, tags, token2idx, tag2idx, max_len=50):
        self.tokens = tokens
        self.tags = tags
        self.token2idx = token2idx
        self.tag2idx = tag2idx
        self.max_len = max_len
    
    def __len__(self):
        return len(self.tokens)
    
    def __getitem__(self, idx):
        token_seq = self.tokens[idx]
        tag_seq = self.tags[idx]
        # Convert tokens to indices, pad if necessary
        token_ids = [self.token2idx.get(tok, self.token2idx["<UNK>"]) for tok in token_seq]
        tag_ids = [self.tag2idx[tag] for tag in tag_seq]
        
        if len(token_ids) < self.max_len:
            pad_length = self.max_len - len(token_ids)
            token_ids.extend([self.token2idx["<PAD>"]]*pad_length)
            tag_ids.extend([self.tag2idx["O"]]*pad_length)
            mask = [1]*len(token_seq) + [0]*pad_length
        else:
            token_ids = token_ids[:self.max_len]
            tag_ids = tag_ids[:self.max_len]
            mask = [1]*self.max_len
        
        return torch.tensor(token_ids), torch.tensor(tag_ids), torch.tensor(mask)




In [36]:
# Example: build token2idx and tag2idx dictionaries based on your dataset.
token2idx = ['I',
  'hate',
  'the',
  'words',
  'chunder']
# Get label names from the dataset features
#label_list = dataset["train"].features["ner_tags"].feature.names

# Mapping from tag to index and back
tag2idx = {tag: idx for idx, tag in enumerate(token2idx)}
idx2tag = {idx: tag for tag, idx in tag2idx.items()}


In [37]:
import torch.optim as optim

# Hyperparameters
vocab_size = len(token2idx)
tagset_size = len(tag2idx)
embedding_dim = 100
hidden_dim = 128
num_epochs = 10
batch_size = 32

# Initialize model, optimizer
model = BiLSTM_CRF(vocab_size, tagset_size, embedding_dim, hidden_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create DataLoader
train_dataset = NERDataset(train_tokens, train_tags, token2idx, tag2idx)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for token_ids, tag_ids, mask in train_loader:
        optimizer.zero_grad()
        loss = model(token_ids, tags=tag_ids, mask=mask)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")


AttributeError: 'list' object has no attribute 'get'

In [38]:
# deepseek start
from datasets import load_dataset
import numpy as np
from collections import Counter

btc=dataset
# Explore dataset structure
print(btc)
print("Sample tweet:", btc["train"][0]["tokens"])
print("Sample tags:", btc["train"][0]["tags"])

# Get tag names
tag_names = btc["train"].features["tags"].feature.names
print("Tag names:", tag_names)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 6338
    })
    validation: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 2000
    })
})
Sample tweet: ['I', 'hate', 'the', 'words', 'chunder', ',', 'vomit', 'and', 'puke', '.', 'BUUH', '.']
Sample tags: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
Tag names: ['B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O']


In [40]:
import re

def preprocess_tweet(tokens):
    processed = []
    for token in tokens:
        # Handle mentions
        if token.startswith("@"):
            token = "@USER"
        # Handle URLs
        elif token.startswith("http"):
            token = "HTTPURL"
        # Normalize repeated characters (e.g., "loooool" -> "loool")
        token = re.sub(r'(.)\1{3,}', r'\1\1\1', token)
        processed.append(token.lower())
    return processed

# Example
sample_tokens = ["@john", "loooool", "check", "this", "http://example.com"]
print(preprocess_tweet(sample_tokens))

['@user', 'loool', 'check', 'this', 'httpurl']


In [41]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        
        # Embedding layer
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        
        # BiLSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        
        # Linear layer to map to tag space
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        
        # CRF layer
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))
        
        # Initialize transitions with constraints
        self.transitions.data[tag_to_ix["O"], tag_to_ix["B-PER"]] = -10000
        # Add more constraints as needed
        
    def forward(self, sentence):
        # Get embeddings
        embeds = self.word_embeds(sentence)
        
        # Get LSTM features
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        
        # Get emission scores
        emissions = self.hidden2tag(lstm_out)
        return emissions

In [43]:
from sklearn.metrics import classification_report
from seqeval.metrics import f1_score

def train_model(model, train_data, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for sentence, tags in train_data:
            # Forward pass
            emissions = model(sentence)
            
            # Compute loss
            loss = model.crf(emissions, tags)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_data)}")

def evaluate_model(model, test_data):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for sentence, tags in test_data:
            emissions = model(sentence)
            predicted_tags = model.crf.decode(emissions)
            
            y_true.extend(tags)
            y_pred.extend(predicted_tags)
    
    print(classification_report(y_true, y_pred, target_names=tag_names))
    print("F1-score:", f1_score(y_true, y_pred))

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

# Load pretrained model
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased", 
    num_labels=len(tag_names),
    id2label={i: label for i, label in enumerate(tag_names)},
    label2id={label: i for i, label in enumerate(tag_names)}
)


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\arpan\anaconda3\envs\text_analytics\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [tokens, tags]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [45]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    remove_unused_columns=False,
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=btc["train"],
    eval_dataset=btc["validation"],
)

# Train
trainer.train()

ValueError: too many dimensions 'str'